In [101]:
# import googlemaps
# import pandas as pd
# from geopy.distance import geodesic
# import os
from dotenv import load_dotenv
import imports as i
import importlib
importlib.reload(i)

<module 'imports' from 'c:\\Users\\Veronii\\Desktop\\data-wild-west\\code\\imports.py'>

In [102]:
load_dotenv()

True

In [103]:
gmaps = i.googlemaps.Client(key=i.os.getenv('api_key'))

In [104]:
cities = ['Copenhagen', 'Aalborg', 'Arhus']
gyms = ['PureGym Copenhagen', 'PureGym Aalborg', 'PureGym Arhus']
loc_name = cities[0]    

place_details = gmaps.places(loc_name)

In [105]:
# function two calculate the distance between two places by using the lat and lng
def distance_calc(lat1, lng1, lat2, lng2):

    distance_calculator = i.geodesic((lat1, lng1), (lat2, lng2))
    distance_in_meters = distance_calculator.meters 

    return distance_in_meters

 ### Finding nearby transportation for Pure Gyms in Copenhagen, Aalborg and Arhus

In [106]:
list = []

for gym in gyms: 
    places_results = gmaps.places(query=gym)

    if places_results['status'] == 'OK' and len(places_results['results']) > 0:
        
        for place in places_results['results']:
            place_id = place['place_id']
            place_details = gmaps.place(place_id)   
            location = place_details['result']['geometry']['location']
            lat = location['lat']
            lng = location['lng']

            # Avaliable transportation close (only bus station, train station and transit station which includes metro)
            trasnportation = ['bus_station', 'train_station', 'transit_station']
            radius = 500 # radius in meters for how much close you want to search for tranportation (we let it 500 meters for now)
            
            for transport_type in trasnportation:
                transport_results = gmaps.places_nearby(location={'lat': lat, 'lng': lng}, radius=radius, type=transport_type)
                
                for transport in transport_results['results']:
                    transport_id = transport['place_id']
                    transport_name = transport['name']
                    transport_location = transport['geometry']['location']
                    transport_lat = transport_location['lat']
                    transport_lng = transport_location['lng']
                    distance = round(distance_calc(lat, lng, transport_lat, transport_lng)) # rounding the value (distance in meters)

                    
                    list.append({
                        'place_id': place_id,
                        'name': gym,
                        'lat': lat,
                        'lng': lng,
                        'transport_id': transport_id,
                        'transport_name': transport_name,
                        'transport_type': transport_type,
                        'transport_lat': transport_lat,
                        'transport_lng': transport_lng,
                        'distance_gym_transport': distance
                    })

df = i.pd.DataFrame(list)
df

,place_id,name,lat,lng,transport_id,transport_name,transport_type,transport_lat,transport_lng,distance_gym_transport
0,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,ChIJ-y92w3VTUkYRY8NOJNuwQkQ,Kridt v/Rikke Frisk,bus_station,55.668036,12.551084,305
1,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,ChIJ76k9hJ9TUkYRRaHYcLRX3XE,Lysholdet v/Jakob Holst,bus_station,55.672786,12.547279,331
2,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,ChIJydMagp9TUkYRjfgBz2sKErQ,"Ejerforeningen Sigbrits Allé 3, 5 og 5a",bus_station,55.672865,12.546713,343
3,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,ChIJydMagp9TUkYRcgSv8LkczNs,Grundejerforeningen Carl Jacobsens Vej 33-41,bus_station,55.672865,12.546713,343
4,ChIJh3mB6UxSUkYREbiH4JDK-7M,PureGym Copenhagen,55.669812,12.547390,ChIJl46JnZ5TUkYRnW93t25gFhQ,Optiperform v/Maja Juel-Hansen,bus_station,55.668011,12.542917,346
...,...,...,...,...,...,...,...,...,...,...
689,ChIJ2ZHhefs_TEYRGLo-pDms87A,PureGym Arhus,56.151658,10.173392,ChIJ12cnkuQ_TEYRvnlNDlHZ4XY,Birkeparken/Silkeborgvej (Aarhus Kom),transit_station,56.155973,10.175228,494
690,ChIJ2ZHhefs_TEYRGLo-pDms87A,PureGym Arhus,56.151658,10.173392,ChIJ-W-y2OI_TEYRLacYXu2WI1c,Åbyhøj Torv/Silkeborgvej (Aarhus Kom),transit_station,56.155508,10.169321,498
691,ChIJAfU9wBIVTEYRJ7DNdJ0UqFA,PureGym Arhus,56.178129,10.139422,ChIJXzDG-RIVTEYRnUkWDFMPe4A,Bredskiftevej/Viborgvej (Aarhus Kom),transit_station,56.177432,10.137240,156
692,ChIJAfU9wBIVTEYRJ7DNdJ0UqFA,PureGym Arhus,56.178129,10.139422,ChIJR5vRXxIVTEYR04QjOMwwLn0,Bredskiftevej/Viborgvej (Aarhus Kom),transit_station,56.178563,10.136740,173


In [91]:
df.to_csv('../data/raw_data/transportation.csv', index=False)